In [23]:
# this line makes figures interactive in Jupyter notebooks
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import cantera as ct

from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity

# for convenience:
def to_si(quant):
    '''Converts a Pint Quantity to magnitude at base SI units.
    '''
    return quant.to_base_units().magnitude

In [24]:
# these lines are only for helping improve the display
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['figure.dpi']= 150
plt.rcParams['savefig.dpi'] = 150

In [88]:
#full_species = {S.name: S for S in ct.Species.listFromFile('nasa_gas.cti')}

#species = [full_species[S] for S in (
#    'N2H4', 'N2', 'H2', 'H', 'N', 'NH'
#    )]
#gas = ct.Solution(thermo='IdealGas', species=species)

#gas = ct.Solution('n2h4.yaml')
gas = ct.Solution('nasa_n2h4.cti')

temperature = Q_(5000, 'K')
pressure = Q_(50, 'psi')

gas.TPX = to_si(temperature), to_si(pressure), 'N2H4:1.0'
#gas.equilibrate('TP')
#gas()
#print(Q_(gas.P, 'Pa').to('bar'))

# based on example https://cantera.org/examples/python/thermo/sound_speed.py.html

# set the gas to equilibrium at its current T and P
rtol=1.0e-6
maxiter=5000
gas.equilibrate('TP', rtol=rtol, max_iter=maxiter)

# save properties
X0 = gas.X
s0 = gas.s
p0 = gas.P
h0 = gas.h
T0 = gas.T
r0 = gas.density

print(f'gamma=cp/cv: {gas.cp/gas.cv}')

# perturb the pressure
p1 = p0*1.0001

# set the gas to a state with the same entropy and composition but
# the perturbed pressure
gas.SP = s0, p1

# frozen sound speed
afrozen = np.sqrt((p1 - p0)/(gas.density - r0))
gamma_frozen = (np.log(p1) - np.log(p0)) / (np.log(gas.density) - np.log(r0))

# now equilibrate the gas holding S and P constant
gas.equilibrate('SP', rtol=rtol, max_iter=maxiter)

# equilibrium sound speed
aequil = np.sqrt((p1 - p0)/(gas.density - r0))
gamma_equil = (np.log(p1) - np.log(p0)) / (np.log(gas.density) - np.log(r0))

# compute the frozen sound speed using the ideal gas expression as a check
gamma = gas.cp/gas.cv
afrozen2 = np.sqrt(gamma * ct.gas_constant * gas.T /
                   gas.mean_molecular_weight)

gas.TPX = T0, p0, X0
T1 = T0 * 1.0001
gas.TP = T1, p0
gas.equilibrate('SP', rtol=rtol, max_iter=maxiter)
cp_equil = (gas.h - h0) / (gas.T - T0)
print(f'C_p,equil = {cp_equil}')

print(gamma_frozen, gamma_equil, gamma)
print(afrozen, aequil, afrozen2)

gamma=cp/cv: 1.4919307570740907
C_p,equil = 11090.553125494333
1.4919186582279218 1.2550230291153577 1.4919329172682332
3048.5785541452924 2796.0780576325237 3048.598968608774


In [94]:
def get_thermo_derivatives(gas):
    '''Gets thermo derivatives based on shifting equilibrium.
    '''
    # unknowns for system with no condensed species:
    # dpi_i_dlogT_P (# elements)
    # dlogn_dlogT_P
    # dpi_i_dlogP_T (# elements)
    # dlogn_dlogP_T
    # total unknowns: 2*n_elements + 2

    num_var = 2 * gas.n_elements + 2

    coeff_matrix = np.zeros((num_var, num_var))
    right_hand_side = np.zeros(num_var)

    tot_moles = 1.0 / gas.mean_molecular_weight
    moles = gas.X * tot_moles

    condensed = False

    # indices
    idx_dpi_dlogT_P = 0
    idx_dlogn_dlogT_P = idx_dpi_dlogT_P + gas.n_elements
    idx_dpi_dlogP_T = idx_dlogn_dlogT_P + 1
    idx_dlogn_dlogP_T = idx_dpi_dlogP_T + gas.n_elements

    # construct matrix of elemental stoichiometric coefficients
    stoich_coeffs = np.zeros((gas.n_elements, gas.n_species))
    for i, elem in enumerate(gas.element_names):
        for j, sp in enumerate(gas.species_names):
            stoich_coeffs[i,j] = gas.n_atoms(sp, elem)

    # equations for derivatives with respect to temperature
    # first n_elements equations
    for k in range(gas.n_elements):
        for i in range(gas.n_elements):
            coeff_matrix[k,i] = np.sum(stoich_coeffs[k,:] * stoich_coeffs[i,:] * moles)
        coeff_matrix[k, gas.n_elements] = np.sum(stoich_coeffs[k,:] * moles)
        right_hand_side[k] = -np.sum(stoich_coeffs[k,:] * moles * gas.standard_enthalpies_RT)

    # skip equation relevant to condensed species

    for i in range(gas.n_elements):
        coeff_matrix[gas.n_elements, i] = np.sum(stoich_coeffs[i, :] * moles)
    right_hand_side[gas.n_elements] = -np.sum(moles * gas.standard_enthalpies_RT)

    # equations for derivatives with respect to pressure

    for k in range(gas.n_elements):
        for i in range(gas.n_elements):
            coeff_matrix[gas.n_elements+1+k,gas.n_elements+1+i] = np.sum(stoich_coeffs[k,:] * stoich_coeffs[i,:] * moles)
        coeff_matrix[gas.n_elements+1+k, 2*gas.n_elements+1] = np.sum(stoich_coeffs[k,:] * moles)
        right_hand_side[gas.n_elements+1+k] = np.sum(stoich_coeffs[k,:] * moles)

    for i in range(gas.n_elements):
        coeff_matrix[2*gas.n_elements+1, gas.n_elements+1+i] = np.sum(stoich_coeffs[i, :] * moles)
    right_hand_side[2*gas.n_elements+1] = np.sum(moles)
    
    derivs = np.linalg.solve(coeff_matrix, right_hand_side)

    dpi_dlogT_P = derivs[idx_dpi_dlogT_P : idx_dpi_dlogT_P + gas.n_elements]
    dlogn_dlogT_P = derivs[idx_dlogn_dlogT_P]
    dpi_dlogP_T = derivs[idx_dpi_dlogP_T]
    dlogn_dlogP_T = derivs[idx_dlogn_dlogP_T]

    # dpi_dlogP_T is not used
    
    return dpi_dlogT_P, dlogn_dlogT_P, dlogn_dlogP_T

def get_thermo_properties(gas, dpi_dlogT_P, dlogn_dlogT_P, dlogn_dlogP_T):
    '''Calculates specific heats, volume derivatives, and specific heat ratio.
    
    Based on shifting equilibrium for mixtures.
    '''
    
    tot_moles = 1.0 / gas.mean_molecular_weight
    moles = gas.X * tot_moles
    
    # construct matrix of elemental stoichiometric coefficients
    stoich_coeffs = np.zeros((gas.n_elements, gas.n_species))
    for i, elem in enumerate(gas.element_names):
        for j, sp in enumerate(gas.species_names):
            stoich_coeffs[i,j] = gas.n_atoms(sp, elem)
    
    spec_heat_p = ct.gas_constant * (
        np.sum([dpi_dlogT_P[i] * 
                np.sum(stoich_coeffs[i,:] * moles * gas.standard_enthalpies_RT) 
                for i in range(gas.n_elements)
                ]) +
        np.sum(moles * gas.standard_enthalpies_RT) * dlogn_dlogT_P +
        np.sum(moles * gas.standard_cp_R) +
        np.sum(moles * gas.standard_enthalpies_RT**2)
        )
    
    dlogV_dlogT_P = 1 + dlogn_dlogT_P
    dlogV_dlogP_T = -1 + dlogn_dlogP_T
    
    spec_heat_v = (
        spec_heat_p + gas.P * gas.v / gas.T * dlogV_dlogT_P**2 / dlogV_dlogP_T
        )

    gamma = spec_heat_p / spec_heat_v
    gamma_s = -gamma/dlogV_dlogP_T
    
    return dlogV_dlogT_P, dlogV_dlogP_T, spec_heat_p, gamma_s

In [95]:
gas = ct.Solution('nasa_n2h4.cti')

temperature = Q_(5000, 'K')
pressure = Q_(50, 'psi')

gas.TPX = to_si(temperature), to_si(pressure), 'N2H4:1.0'

# set the gas to equilibrium at its current T and P
rtol=1.0e-6
maxiter=5000
gas.equilibrate('TP', rtol=rtol, max_iter=maxiter)

In [96]:
derivs = get_thermo_derivatives(gas)

dlogV_dlogT_P, dlogV_dlogP_T, cp, gamma_s = get_thermo_properties(
    gas, derivs[0], derivs[1], derivs[2]
    )

print(f'Cp = {cp: .2f}')

print(f'(d log V/d log P)_T = {dlogV_dlogP_T: .4f}')
print(f'(d log V/d log T)_P = {dlogV_dlogT_P: .4f}')

print(f'gamma_s = {gamma_s: .4f}')

Cp =  11091.25
(d log V/d log P)_T = -1.0399
(d log V/d log T)_P =  1.4713
gamma_s =  1.2551


In [97]:
speed_sound = np.sqrt(ct.gas_constant * gas.T * gamma_s / gas.mean_molecular_weight)
print(f'Speed of sound = {speed_sound: .1f} m/s')

Speed of sound =  2796.1 m/s


In [116]:
# example: adiabatic combustion chamber temperature

temperature = Q_(90, 'K')
pressure = Q_(20, 'MPa')

gas2 = ct.Solution('h2o2.cti')
gas2.TPX = to_si(temperature), to_si(pressure), 'O2:0.5, H2:1.0'
gas2.equilibrate('HP')
gas2()


  ohmech:

       temperature   3785.4 K
          pressure   2e+07 Pa
           density   10.288 kg/m^3
  mean mol. weight   16.19 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -4.8264e+05        -7.814e+06  J
   internal energy       -2.4267e+06       -3.9288e+07  J
           entropy             15262        2.4709e+05  J/K
    Gibbs function       -5.8255e+07       -9.4314e+08  J
 heat capacity c_p            3280.1             53104  J/K
 heat capacity c_v            2766.5             44789  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2          0.013417           0.10775            -18.64
                 H         0.0015639          0.025118           -9.3201
                 O           0.01191          0.012052           -14.704
  

In [126]:
full_species = {S.name: S for S in ct.Species.listFromFile('nasa_gas.cti')}

species = [full_species[S] for S in (
    'N2H4', 'N2', 'H2', 'H', 'N', 'NH'
    )]
gas = ct.Solution(thermo='IdealGas', species=species)

temperature = Q_(5000, 'K')
pressure = Q_(50, 'psi')

gas.TPX = to_si(temperature), to_si(pressure), 'N2H4:1.0'

# set the gas to equilibrium at its current T and P
gas.equilibrate('TP')

derivs = get_thermo_derivatives(gas)

dlogV_dlogT_P, dlogV_dlogP_T, cp, gamma_s = get_thermo_properties(
    gas, derivs[0], derivs[1], derivs[2]
    )

print(f'Cp = {cp: .2f}')

print(f'(d log V/d log P)_T = {dlogV_dlogP_T: .4f}')
print(f'(d log V/d log T)_P = {dlogV_dlogT_P: .4f}')

print(f'gamma_s = {gamma_s: .4f}')

speed_sound = np.sqrt(ct.gas_constant * gas.T * gamma_s / gas.mean_molecular_weight)
print(f'Speed of sound = {speed_sound: .1f} m/s')

gas()

Cp =  11104.47
(d log V/d log P)_T = -1.0400
(d log V/d log T)_P =  1.4722
gamma_s =  1.2549
Speed of sound =  2795.8 m/s

       temperature   5000 K
          pressure   3.4474e+05 Pa
           density   0.055346 kg/m^3
  mean mol. weight   6.6743 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        4.2088e+07        2.8091e+08  J
   internal energy         3.586e+07        2.3934e+08  J
           entropy             29182        1.9477e+05  J/K
    Gibbs function       -1.0382e+08       -6.9294e+08  J
 heat capacity c_p            3779.4             25225  J/K
 heat capacity c_v            2533.6             16910  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                N2            0.8567           0.20411           -30.731
                H2          0.0

In [124]:
o_f_ratio = 6.0

h2 = ct.Solution('test.yaml', 'liquid_hydrogen')
h2.TP = 20.270, 20477430

o2 = ct.Solution('test.yaml', 'liquid_oxygen')
o2.TP = 90.170, 20477430

molar_ratio = o_f_ratio / (o2.mean_molecular_weight / h2.mean_molecular_weight)
moles_ox = molar_ratio / (1 + molar_ratio)
moles_f = 1 - moles_ox

gas3 = ct.Solution('nasa_h2o2.cti', 'gas')

#full_species = {S.name: S for S in ct.Species.listFromFile('h2o2.yaml')}
#species = [full_species[S] for S in full_species if S != 'AR']
#gas3 = ct.Solution(thermo='IdealGas', species=species)

mix = ct.Mixture([(h2, moles_f), (o2, moles_ox), (gas3, 0)])
mix.equilibrate('HP')

gas3()


  gas:

       temperature   3596.6 K
          pressure   2.0477e+07 Pa
           density   9.3213 kg/m^3
  mean mol. weight   13.612 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -9.8628e+05       -1.3425e+07  J
   internal energy       -3.1831e+06       -4.3329e+07  J
           entropy             17181        2.3387e+05  J/K
    Gibbs function        -6.278e+07       -8.5455e+08  J
 heat capacity c_p            3795.3             51661  J/K
 heat capacity c_v            3184.5             43347  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                 H         0.0018958          0.025601           -8.7962
               HO2        8.3374e-05        3.4384e-05           -40.632
                H2          0.036637           0.24737           -17.592

In [125]:
print(gas3.species_names)
derivs = get_thermo_derivatives(gas3)

dlogV_dlogT_P, dlogV_dlogP_T, cp, gamma_s = get_thermo_properties(
    gas3, derivs[0], derivs[1], derivs[2]
    )

print(f'Cp = {cp: .2f}')

print(f'(d log V/d log P)_T = {dlogV_dlogP_T: .4f}')
print(f'(d log V/d log T)_P = {dlogV_dlogT_P: .4f}')

print(f'gamma_s = {gamma_s: .4f}')

speed_sound = np.sqrt(ct.gas_constant * gas3.T * gamma_s / gas3.mean_molecular_weight)
print(f'Speed of sound = {speed_sound: .1f} m/s')

['H', 'HO2', 'H2', 'H2O', 'H2O2', 'O', 'OH', 'O2', 'O3']
Cp =  7341.98
(d log V/d log P)_T = -1.0191
(d log V/d log T)_P =  1.3315
gamma_s =  1.1473
Speed of sound =  1587.6 m/s


In [84]:
P = Q_(2.0477e7, 'Pa').to('bar')
print(P)

204.77 bar
